# Harmonization Pipeline Development
This temporary notebook is being used to develop the python and R pipeline used for harmonizing multisite MRI data.

## Import required modules

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np

In [81]:
# Set up ipywidgets for interactive chart.
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import interact, fixed

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## Function Definitions

### Function for Effect Size Calculation

In [82]:
# Function to compute effect sizes
# Based on method in Nakagawa, S. and I.C. Cuthill. (2007). Biol. Rev. 82. pp. 591-605.

import math

def cohensd(t, df, n1, n2):
    d = ( t * (n1+n2) ) / (math.sqrt(n1*n2) * math.sqrt(df))
    se = math.sqrt( ((n1+n2-1)/(n1+n2-3)) * ( (4/(n1+n2)) * (1 + (d**2)/8) ))
    return {'d': d, 'se': se, 'lower_ci': d-1.96*se, 'upper_ci': d+1.96*se}

### Functions to Visualize the Data

In [83]:
# Plot distributions of the data by subset. These are meant to be overlaid, so just show the KDE.
def plot_distributions(data, subset_col, display_col):
    for sub in data[subset_col].unique():
        subset=data[data[subset_col] == sub]
        sns.distplot(subset[display_col],hist=False, kde=True, label=sub)

In [84]:
# Plot distributions, using stacked violin plots, split by the values in y.
def plot_stacked_dists(x, y, data, scale = 'count'):
    # Plot the distributions of ages across all sites. Extra hue_class column is a hacky solution to get seaborn 
    # to display half a violin plot.
    plt.figure()
    sns.set(style = 'whitegrid')
    vp_data = pd.DataFrame({'idx': data.index, x: data[x], y: data[y], 'hue_class': 0})
    vp_data['hue_class'].loc[-1] = 999
    ax_vp = sns.violinplot(y = y, x = x, data = vp_data, hue = 'hue_class', split = True, scale = scale)
    ax_vp.legend().remove()
    plt.show()

In [85]:
# Function to plot original and harmonized distributions together, by site.
def plot_dual_distributions(x, y, data_unharmonized, data_harmonized, scale = 'count'):
    # Plots two distibutions (e.g. harmonized and unharmonized data) as the two halves of stacked violin plots.
    plt.figure(figsize = (12,8))
    vp_data = pd.DataFrame({x: data_unharmonized[x], y: data_unharmonized[y], 'Legend': 'Unharmonized'})
    vp_data = vp_data.append(pd.DataFrame({x: data_harmonized[x], y: data_harmonized[y], 'Legend': 'Harmonized'}))
    ax_h_uh = sns.violinplot(y = y, x = x, data = vp_data, hue = 'Legend', split = True, scale = scale)
    #ax_h_uh.legend().remove()
    plt.show()

In [86]:
# Plot a panel of figures useful for quick data exploration and verification.
def plot_panel(nucleus, data1, data2):
    plt.figure()
    fig, axes = plt.subplots(ncols=2, nrows = 2, figsize = (13,4))
    plt.subplots_adjust(wspace = 0.5)
    
    # Plot nucleus volume by Age and TBV: data1
    sns.scatterplot('Age', nucleus, data = data1, hue = "DX", ax=axes[0,0])
    sns.regplot('Age', nucleus, data = data1, scatter = False, ax=axes[0,0])
    sns.scatterplot('TBV', nucleus, data = data1, hue = "DX", ax=axes[0,1])
    sns.regplot('TBV', nucleus, data = data1, scatter = False, ax=axes[0,1])
    
    # Plot nucleus volume by Age and TBV: data2
    sns.scatterplot('Age', nucleus, data = data2, hue = "DX", ax=axes[1,0])
    sns.regplot('Age', nucleus, data = data2, scatter = False, ax=axes[1,0])
    sns.scatterplot('TBV', nucleus, data = data2, hue = "DX", ax=axes[1,1])
    sns.regplot('TBV', nucleus, data = data2, scatter = False, ax=axes[1,1])
    
    # Plot the two distributions together
    plot_dual_distributions(x=nucleus, y = 'Site', data_unharmonized = data1, data_harmonized = data2, scale = 'count')

In [87]:
# Function to plot fitted values against residuals
# This relies on internal structure of the models. For now, it's the same for the linear regression and linear mixed models.
#def plot_fitted_vs_residual(models, ex_data, x_fitted_col, y_resid_col, display_col, hue_col, title_add = None):
def plot_fitted_vs_residual(models, ex_data, display_col, hue_col, title_add = None):
    plt.figure(figsize = (12,8))
    fitted_y = models[display_col].fittedvalues
    resid = models[display_col].resid
    # The following is not ideal. Need to revisit this solution.
    fit_resid_df = pd.DataFrame({'Residual': resid, 'Fitted_Value': fitted_y, 'Site': ex_data['Site'], 'DX': ex_data['DX']})
    ax = sns.scatterplot(fitted_y, resid, hue = hue_col, data=fit_resid_df)
    ax.set_title('Fitted vs. Residual Plot for ' + display_col + title_add)
    ax.set(xlabel = 'Fitted', ylabel = 'Residual')
    ax.legend().remove()
    plt.show()  

## Read data and drop rows with missing values for age, examine structure

In [88]:
data = pd.read_csv("../data/data_trimmed_allsites.csv", index_col=0)
#data = pd.read_csv("../data/data_trimmed.csv", index_col=0)

# Age is missing in some rows, but is needed for modelling. Drop those rows
complete_rows = data['Age'].notna()
NA_rows = ~complete_rows
data = data[complete_rows]

## Mask features that failed segmentation QC

In [89]:
# QC columns are of the form L_str. Volume data is in L_str_vol
QC_cols = ['L_str', 'L_GP', 'L_thal', 'R_str', 'R_GP', 'R_thal']
features = [QCc + '_vol' for QCc in QC_cols]

QC_failed = data[QC_cols] <= 0.5
data[features] = data[features].mask(QC_failed.values)

# Write out data with failed structures masked
data.to_csv('intermediate_data_masked.csv')

## Get harmonized subcortical volume data from neuroCombat

In [90]:
# Read in harmonized features from R output
harmonized_features = pd.read_csv("harmonized_masked.csv", index_col=0)

### Determine Effect of Diagnosis: Linear Models with ComBat Harmonized Data


In [91]:
# Linear regression on ComBat harmonized data
from statsmodels.formula.api import ols

models = pd.Series(dtype='object')
model_params = pd.Series(dtype = 'object')
model_es = []

for nucleus in harmonized_features.columns[:-5]:        # Don't include last (covariate) columns
    models[nucleus] = ols(formula = nucleus + ' ~ DX + Age + Sex + TBV', data = harmonized_features).fit()
#    print(models[nucleus].summary())
    model_params[nucleus] = models[nucleus].params

    # Get effect size, being sure to drop rows with NA for age.
    n_control = sum(data['DX'] == 'Control')
    n_ASD = models[nucleus].nobs - n_control
    model_es.append(cohensd(t  = models[nucleus].tvalues['DX[T.Control]'], 
                                df = models[nucleus].df_resid, 
                                n1 = n_control,
                                n2 = n_ASD))

combat_es = pd.DataFrame(model_es, index=features)
combat_es_filename = "intermediate_es_combat.csv"
es_filenames = [combat_es_filename]
es_names = ["Combat"]
combat_es.to_csv(combat_es_filename)

## Compute Linear Regression on Unharmonized Data

In [92]:
# Linear regression on unharmonized data
models_unh = pd.Series(dtype='object')
model_unh_params = pd.Series(dtype = 'object')
model_unh_es = []

for nucleus in features:
    models_unh[nucleus] = ols(formula = nucleus + ' ~ DX + Age + Sex + TBV + Site', data = data).fit()
#    print(models_unh[nucleus].summary())
    model_unh_params[nucleus] = models_unh[nucleus].params
    
    # Get effect size, being sure to drop rows with NA for age.
    n_control = sum(data['DX'] == 'Control')
    n_ASD = models_unh[nucleus].nobs - n_control
    model_unh_es.append(cohensd(t  = models_unh[nucleus].tvalues['DX[T.Control]'], 
                                    df = models_unh[nucleus].df_resid, 
                                    n1 = n_control,
                                    n2 = n_ASD))

unh_es = pd.DataFrame(model_unh_es, index=features)
unh_es_filename = "intermediate_es_unh.csv"
es_filenames.append(unh_es_filename)
es_names.append("Unharmonized")
unh_es.to_csv(unh_es_filename)

## Linear Mixed Model on the Raw Data
An alternative analysis is to run a linear mixed model on the raw (unharmonized) data, with site as a random effect. 

In [93]:
# Code in this cell based on https://www.statsmodels.org/stable/mixed_linear.html
# and https://www.statsmodels.org/stable/examples/notebooks/generated/mixed_lm_example.html
# import statsmodels.api as sm
import statsmodels.formula.api as smf

lm_models = pd.Series(dtype='object')
lm_model_params = pd.Series(dtype = 'object')
lm_model_es = []

for nucleus in features:
    lm_models[nucleus] = smf.mixedlm(nucleus + " ~ DX + Age + Sex + TBV", data, groups = data["Site"], missing='drop').fit()
    lm_model_params[nucleus] = lm_models[nucleus].params
    
    # Get effect size, being sure to drop rows with NA for age.
    n_control = sum(data['DX'] == 'Control')
    n_ASD = lm_models[nucleus].nobs - n_control
    lm_model_es.append(cohensd(t  = lm_models[nucleus].tvalues['DX[T.Control]'], 
                          df = lm_models[nucleus].df_resid, 
                          n1 = n_control,
                          n2 = n_ASD))
    
lmm_es = pd.DataFrame(lm_model_es, index=features)
lmm_es_filename = "intermediate_es_lmm.csv"
es_filenames.append(lmm_es_filename)
es_names.append("Mixed_model")
lmm_es.to_csv(lmm_es_filename)

## Collect Effect Sizes and Export for Forest Plot in R

In [94]:
# Write out files containing output file names and type of data
pd.DataFrame(es_filenames).to_csv("es_filenames.csv", index=False, header=False)
pd.DataFrame(es_names).to_csv("es_names.csv", index=False, header=False)


# To Do
 1. Add references section
 2. Look up (Ross's? Greg's?) documentation strings and document functions
 3. Deal with missing values, and remove values based on QC.
 4. Make the code more "Pythonic" - there are places where I'm not using dataframes and series in quite the optimal way.
 5. Add meta-analysis code for another option - it's another ENIGMA technique, with mature packages available in R, but not in Python.
 6. Create an interactive figure that shows the effect of adding or removing terms from the model. Noticed some significant clustering in the residual vs. fitted plots by DX when TBV was taken out of the model. When added back in, the clustering disappeared. This was particularly evident in the thalamus.
 7. Check structures against TBV, and DX against TBV. The clustering in the thalamus (see above) was quite noticeable, and disappears when TBV is included in the model. Note that p values for DX move from ~ .05 to > .5 when TBV is included in the model. This suggests that any volumetric effect in the subcortical structures is largely driven by differences in TBV.
 * Consider, should I be harmonizing TBV as well?
 8. Plot the effect sizes and SEs.
 9. Open data, and package up with a full data file.
 10. Deal with collinearity in models, e.g. nucleus volume and TBV.

## In the future
 1. Have a look at metafor - can I reproduce some of it in a Python package? That would be a cool way to make a contribution. Could do it piecemeal. Discuss with Gabe.
 